# Libraries

In [ ]:
# Importing necessary modules
import os
import gc
import sys
import math
import time
import random
import datetime as dt
import numpy as np
import pandas as pd
import wandb

from glob import glob
from pathlib import Path
from typing import Dict, List, Union
import scipy.signal as scisig
from scipy.signal import butter, lfilter, freqz
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

# Importing PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import (
    ReduceLROnPlateau,
    OneCycleLR,
    CosineAnnealingLR,
    CosineAnnealingWarmRestarts,
)
from torch.optim.optimizer import Optimizer
from sklearn.model_selection import GroupKFold

# Adding path for Kaggle kernel
sys.path.append("/kaggle/input/kaggle-kl-div")
import kaggle_kl_div

# Ignoring warnings
import warnings
warnings.filterwarnings("ignore")

# Setting device to CUDA
device = torch.device("cuda")
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Printing system information
!cat /etc/os-release | grep -oP "PRETTY_NAME=\"\K([^\"]*)"
print(f"BUILD_DATE={os.environ['BUILD_DATE']}, CONTAINER_NAME={os.environ['CONTAINER_NAME']}")

try:
    print(f"PyTorch Version:{torch.__version__}, CUDA is available:{torch.cuda.is_available()}, Version CUDA:{torch.version.cuda}")
    print(f"Device Capability:{torch.cuda.get_device_capability()}, {torch.cuda.get_arch_list()}")
    print(f"CuDNN Enabled:{torch.backends.cudnn.enabled}, Version:{torch.backends.cudnn.version()}")
except Exception:
    pass

# Directory settings

In [ ]:
class APP:
    # Check if running in Jupyter notebook
    jupyter = "ipykernel" in globals()
    if not jupyter:
        try:
            if "IPython" in globals().get("__doc__", ""):
                jupyter = True
        except Exception as inst:
            print(inst)

    # Check if running in Kaggle kernel
    kaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "") != ""
    
    # Check if running locally
    local = os.environ.get("DOCKER_USING", "") == "LOCAL"
    
    # Get the current date and time
    date_time_start = dt.datetime.now()
    dt_start_ymd_hms = date_time_start.strftime("%Y.%m.%d_%H-%M-%S")

    # Get the file run path
    file_run_path = ""
    if jupyter:
        try:
            file_run_path = Path(globals().get("__vsc_ipynb_file__", ""))
        except Exception as inst:
            print(inst)
    else:
        try:
            file_run_path = Path(__file__)
        except Exception as inst:
            print(inst)

    # Get the file run name, app path, run path, and output path
    file_run_name = file_run_path.stem
    path_app = file_run_path.parent
    path_run = Path(os.getcwd())
    path_out = (
        Path("/kaggle/working")
        if kaggle
        else file_run_path / f"{file_run_name}_{dt_start_ymd_hms}"
    )

# Set the output directory
OUTPUT_DIR = "./"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Print the environment information
print(f"jupyter:{APP.jupyter}, kaggle:{APP.kaggle}, local:{APP.local}")
print(APP.file_run_path)
print(APP.path_out)

# Config

In [ ]:
# This is a configuration class for the project
# It contains various parameters and settings for the model training process

class CFG:
    VERSION = '86'  # Version of the project

    wandb = False  # Flag for using Weights and Biases for experiment tracking
    debug = False  # Flag for running in debug mode
    create_eegs = False  # Flag for creating EEGs
    apex = True  # Flag for using Apex for mixed precision training
    visualize = False  # Flag for visualizing data
    save_all_models = True  # Flag for saving all models during training

    # Set the number of workers for data loading and parallel processing
    if debug:
        num_workers = 0
        parallel = False
    else:
        num_workers = os.cpu_count()
        parallel = True

    model_name = "resnet1d_gru"  # Name of the model architecture
    optimizer = "AdamW"  # Optimizer for training

    factor = 0.9  # Factor for reducing learning rate
    eps = 1e-6  # Epsilon value for numerical stability
    lr = 8e-3  # Learning rate
    min_lr = 1e-6  # Minimum learning rate

    batch_size = 64  # Batch size for training
    batch_koef_valid = 2  # Batch size coefficient for validation
    batch_scheduler = True  # Flag for using batch scheduler
    weight_decay = 1e-2  # Weight decay for regularization
    gradient_accumulation_steps = 1  # Number of gradient accumulation steps
    max_grad_norm = 1e7  # Maximum gradient norm

    fixed_kernel_size = 5  # Fixed kernel size for the model
    linear_layer_features = 304  # Number of features in the linear layer
    kernels = [3, 5, 7, 9, 11]  # List of kernel sizes

    seq_length = 50  # Length of the sequence in seconds
    sampling_rate = 200  # Sampling rate in Hz
    nsamples = seq_length * sampling_rate  # Total number of samples
    n_split_samples = 5  # Number of split samples
    out_samples = nsamples // n_split_samples  # Number of output samples
    sample_delta = nsamples - out_samples  # Sample delta
    sample_offset = sample_delta // 2  # Sample offset
    multi_validation = False  # Flag for multi-validation

    train_by_stages = False  # Flag for training by stages
    train_by_folds = True  # Flag for training by folds

    # Define the training stages and corresponding parameters
    n_stages = 2
    match n_stages:
        case 1:
            train_stages = [0]
            epochs = [100]
            test_total_eval = 2
            total_evals_old = [[(2, 3), (6, 29)]]
            total_evaluators = [
                [
                    {'band':(2, 2), 'excl_evals':[]},
                    {'band':(6, 28), 'excl_evals':[]},
                ],
            ]
        case 2:
            train_stages = [0, 1]
            epochs = [50, 100]
            test_total_eval = 0
            total_evals_old = [[(1, 2),(4, 5)], (6, 29)]
            total_evaluators = [
                [
                    {'band':(3, 3), 'excl_evals':[]},
                    {'band':(6, 28), 'excl_evals':[]},
                ],
                [
                    {'band':(1, 2), 'excl_evals':[]},
                    {'band':(4, 5), 'excl_evals':[]},
                ],
            ]
        case 3:
            train_stages = [0, 1, 2]
            epochs = [20, 50, 100]
            test_total_eval = 0
            total_evals_old = [(0, 3), (3, 6), (6, 29)]
            total_evaluators = [
                [
                    {'band':(0, 2), 'excl_evals':[]},
                ],
                [
                    {'band':(3, 5), 'excl_evals':[]},
                ],
                [
                    {'band':(6, 28), 'excl_evals':[]},
                ],
            ]

    n_fold = 5  # Number of folds for cross-validation
    train_folds = [0, 1, 2, 3, 4]  # List of train folds

    patience = 11  # Patience for early stopping
    seed = 2024  # Random seed

    bandpass_filter = {"low": 0.5, "high": 20, "order": 2}  # Bandpass filter parameters
    rand_filter = {"probab": 0.1, "low": 10, "high": 20, "band": 1.0, "order": 2}  # Random filter parameters
    freq_channels = []  # Frequency channels
    filter_order = 2  # Filter order

    random_divide_signal = 0.05  # Random divide signal
    random_close_zone = 0.05  # Random close zone
    random_common_negative_signal = 0.0  # Random common negative signal
    random_common_reverse_signal = 0.0  # Random common reverse signal
    random_negative_signal = 0.05  # Random negative signal
    random_reverse_signal = 0.05  # Random reverse signal

    log_step = 100  # Log step
    log_show = False  # Flag for showing logs

    scheduler = "CosineAnnealingWarmRestarts"  # Scheduler for learning rate

    # Parameters for CosineAnnealingLR scheduler
    cosanneal_params = {
        "T_max": 6,
        "eta_min": 1e-5,
        "last_epoch": -1,
    }

    # Parameters for ReduceLROnPlateau scheduler
    reduce_params = {
        "mode": "min",
        "factor": 0.2,
        "patience": 4,
        "eps": 1e-6,
        "verbose": True,
    }

    # Parameters for CosineAnnealingWarmRestarts scheduler
    cosanneal_res_params = {
        "T_0": 20,
        "eta_min": 1e-6,
        "T_mult": 1,
        "last_epoch": -1,
    }

    # Target columns
    target_cols = [
        "seizure_vote",
        "lpd_vote",
        "gpd_vote",
        "lrda_vote",
        "grda_vote",
        "other_vote",
    ]  

    pred_cols = [x + "_pred" for x in target_cols]  # Predicted columns
    
    # Map features
    map_features = [
        ("Fp1", "T3"),
        ("T3", "O1"),
        ("Fp1", "C3"),
        ("C3", "O1"),
        ("Fp2", "C4"),
        ("C4", "O2"),
        ("Fp2", "T4"),
        ("T4", "O2"),
    ]  

    eeg_features = ["Fp1", "T3", "C3", "O1", "Fp2", "C4", "T4", "O2"]  # EEG features
    feature_to_index = {x: y for x, y in zip(eeg_features, range(len(eeg_features)))}  # Feature to index mapping
    simple_features = []  # Simple features
    n_map_features = len(map_features)  # Number of map features
    in_channels = n_map_features + n_map_features * len(freq_channels) + len(simple_features)  # Number of input channels
    target_size = len(target_cols)  # Size of the target

    # Define the file paths
    path_inp = Path("/kaggle/input")
    path_src = path_inp / "hms-harmful-brain-activity-classification/"
    file_train = path_src / "train.csv"
    path_train = path_src / "train_eegs"
    file_features_test = path_train / "100261680.parquet"
    file_eeg_specs = path_inp / "eeg-spectrogram-by-lead-id-unique/eeg_specs.npy"
    file_raw_eeg = path_inp / "brain-eegs/eegs.npy"

    if APP.kaggle:
        num_workers = 2
        parallel = True

# Print the feature to index mapping and the EEG features
print(CFG.feature_to_index)
print(CFG.eeg_features)

# Utils

In [ ]:
# Initialize the logger for logging messages
def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()


# Function to convert seconds to minutes and seconds format
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


# Function to calculate the time remaining
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))


# Function to quantize data using mu-law encoding
def quantize_data(data, classes):
    mu_x = mu_law_encoding(data, classes)
    return mu_x  # quantized


# Function to perform mu-law encoding
def mu_law_encoding(data, mu):
    mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
    return mu_x


# Function to perform mu-law expansion
def mu_law_expansion(data, mu):
    s = np.sign(data) * (np.exp(np.abs(data) * np.log(mu + 1)) - 1) / mu
    return s


# Function to create a Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype="band")


# Function to apply a Butterworth bandpass filter to data
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Function to apply a Butterworth lowpass filter to data
def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=CFG.sampling_rate, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype="low", analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data


# Function to denoise the data using a bandpass filter
def denoise_filter(x):
    y = butter_bandpass_filter(x, CFG.lowcut, CFG.highcut, CFG.sampling_rate, order=6)
    y = (y + np.roll(y, -1) + np.roll(y, -2) + np.roll(y, -3)) / 4
    y = y[0:-1:4]
    return y

# Parquet to EEG Signals Numpy Processing

In [ ]:
def eeg_from_parquet(
    parquet_path: str, display: bool = False, seq_length=CFG.seq_length
) -> np.ndarray:
    # Read the EEG data from the parquet file
    eeg = pd.read_parquet(parquet_path, columns=CFG.eeg_features)
    rows = len(eeg)

    # Calculate the offset for selecting a subset of data
    offset = (rows - CFG.nsamples) // 2

    # Select a subset of data based on the offset and CFG.nsamples
    eeg = eeg.iloc[offset : offset + CFG.nsamples]

    if display:
        # Display the EEG data
        plt.figure(figsize=(10, 5))
        offset = 0

    # Create an empty array to store the processed EEG data
    data = np.zeros((CFG.nsamples, len(CFG.eeg_features)))

    for index, feature in enumerate(CFG.eeg_features):
        # Get the values of the current feature
        x = eeg[feature].values.astype("float32")

        # Calculate the mean and percentage of NaN values in the feature
        mean = np.nanmean(x)
        nan_percentage = np.isnan(x).mean()

        if nan_percentage < 1:
            # Replace NaN values with the mean value
            x = np.nan_to_num(x, nan=mean)
        else:
            # If all values are NaN, set all values to 0
            x[:] = 0

        # Store the processed feature data in the data array
        data[:, index] = x

        if display:
            if index != 0:
                offset += x.max()
            # Plot the feature data
            plt.plot(range(CFG.nsamples), x - offset, label=feature)
            offset -= x.min()

    if display:
        # Display the plot
        plt.legend()
        name = parquet_path.split("/")[-1].split(".")[0]
        plt.yticks([])
        plt.title(f"EEG {name}", size=16)
        plt.show()

    # Return the processed EEG data
    return data

# Dataset

In [ ]:
class EEGDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        batch_size: int,
        eegs: Dict[int, np.ndarray],
        mode: str = "train",
        downsample: int = None,
        bandpass_filter: Dict[str, Union[int, float]] = None,
        rand_filter: Dict[str, Union[int, float]] = None,
    ):
        self.df = df
        self.batch_size = batch_size
        self.mode = mode
        self.eegs = eegs
        self.downsample = downsample
        self.offset = None
        self.bandpass_filter = bandpass_filter
        self.rand_filter = rand_filter
        
    def __len__(self):
        """
        Length of dataset.
        """
        return len(self.df)

    # Function to get one item from the dataset
    def __getitem__(self, index):
        """
        Get one item.
        """
        X, y_prob = self.__data_generation(index)
        if self.downsample is not None:
            X = X[:: self.downsample, :]
        output = {
            "eeg": torch.tensor(X, dtype=torch.float32),
            "labels": torch.tensor(y_prob, dtype=torch.float32),
        }
        return output

    # Function to set the offset
    def set_offset(self, offset: int):
        self.offset = offset

    # Function to generate data
    def __data_generation(self, index):
        """
        This function generates data for training or testing the model.

        Args:
            index: The index of the data sample to generate.

        Returns:
            A tuple containing the generated data (X) and target probabilities (y_prob).
        """

        # Initialize output array with zeros
        X = np.zeros((CFG.out_samples, CFG.in_channels), dtype="float32")  # Size=(10000, 14)
        random_divide_signal = False

        # Get the data for the specific index
        row = self.df.iloc[index]
        data = self.eegs[row.eeg_id]  # Size=(10000, 8)

        # Handle potential difference between requested and available samples
        if CFG.nsamples != CFG.out_samples:
            if self.mode == "train":
                # Introduce random offset in training mode
                offset = (CFG.sample_delta * random.randint(0, 1000)) // 1000
            elif not self.offset is None:
                # Use predefined offset if available
                offset = self.offset
            else:
                # Use default offset
                offset = CFG.sample_offset

            # Randomly divide signal in training mode with some probability
            if self.mode == "train" and CFG.random_divide_signal > 0.0 and random.uniform(0.0, 1.0) <= CFG.random_divide_signal:
                random_divide_signal = True
                multipliers = [(1, 2), (2, 3), (3, 4), (3, 5)]
                koef_1, koef_2 = multipliers[random.randint(0, 3)]
                offset = (koef_1 * offset) // koef_2
                data = data[offset:offset+(CFG.out_samples * koef_2) // koef_1,:]
            else:
                # Select data with specified offset and number of samples
                data = data[offset:offset+CFG.out_samples,:]

        reverse_signal = False
        negative_signal = False
        if self.mode == "train":
            if CFG.random_common_reverse_signal > 0.0 and random.uniform(0.0, 1.0) <= CFG.random_common_reverse_signal:
                reverse_signal = True
            if CFG.random_common_negative_signal > 0.0 and random.uniform(0.0, 1.0) <= CFG.random_common_negative_signal:
                negative_signal = True

        # Loop through feature pairs defined in configuration
        for i, (feat_a, feat_b) in enumerate(CFG.map_features):
            # Skip this feature pair with some probability in training mode
            if self.mode == "train" and CFG.random_close_zone > 0.0 and random.uniform(0.0, 1.0) <= CFG.random_close_zone:
                continue
            
            # Calculate difference between features
            diff_feat = (data[:, CFG.feature_to_index[feat_a]] - data[:, CFG.feature_to_index[feat_b]])  # Size=(10000,)

            # Apply random reverse or negation in training mode
            if self.mode == "train":
                if reverse_signal or CFG.random_reverse_signal > 0.0 and random.uniform(0.0, 1.0) <= CFG.random_reverse_signal:
                    diff_feat = np.flip(diff_feat)
                if negative_signal or CFG.random_negative_signal > 0.0 and random.uniform(0.0, 1.0) <= CFG.random_negative_signal:
                    diff_feat = -diff_feat

            # Apply pre-defined bandpass filter if available
            if not self.bandpass_filter is None:
                diff_feat = butter_bandpass_filter(
                    diff_feat,
                    self.bandpass_filter["low"],
                    self.bandpass_filter["high"],
                    CFG.sampling_rate,
                    order=self.bandpass_filter["order"],
                )
            
            # Handle random signal division
            if random_divide_signal:
                #diff_feat = cp.asnumpy(cpsig.upfirdn([1.0, 1, 1.0], diff_feat, 2, 3))  # linear interp, rate 2/3
                diff_feat = scisig.upfirdn([1.0, 1, 1.0], diff_feat, koef_1, koef_2)  # linear interp, rate 2/3
                diff_feat = diff_feat[0:CFG.out_samples]

            if (
                self.mode == "train"
                and not self.rand_filter is None
                and random.uniform(0.0, 1.0) <= self.rand_filter["probab"]
            ):
                lowcut = random.randint(
                    self.rand_filter["low"], self.rand_filter["high"]
                )
                highcut = lowcut + self.rand_filter["band"]
                diff_feat = butter_bandpass_filter(
                    diff_feat,
                    lowcut,
                    highcut,
                    CFG.sampling_rate,
                    order=self.rand_filter["order"],
                )

            X[:, i] = diff_feat

        n = CFG.n_map_features
        if len(CFG.freq_channels) > 0:
            for i in range(CFG.n_map_features):
                diff_feat = X[:, i]
                # Apply bandpass filter for each frequency channel
                for j, (lowcut, highcut) in enumerate(CFG.freq_channels):
                    band_feat = butter_bandpass_filter(
                        diff_feat, lowcut, highcut, CFG.sampling_rate, order=CFG.filter_order,  # 6
                    )
                    X[:, n] = band_feat
                    n += 1

        for spml_feat in CFG.simple_features:
            feat_val = data[:, CFG.feature_to_index[spml_feat]]

            # Apply bandpass filter if available
            if not self.bandpass_filter is None:
                feat_val = butter_bandpass_filter(
                    feat_val,
                    self.bandpass_filter["low"],
                    self.bandpass_filter["high"],
                    CFG.sampling_rate,
                    order=self.bandpass_filter["order"],
                )

            # Apply random bandpass filter in training mode with some probability
            if (
                self.mode == "train"
                and not self.rand_filter is None
                and random.uniform(0.0, 1.0) <= self.rand_filter["probab"]
            ):
                lowcut = random.randint(
                    self.rand_filter["low"], self.rand_filter["high"]
                )
                highcut = lowcut + self.rand_filter["band"]
                feat_val = butter_bandpass_filter(
                    feat_val,
                    lowcut,
                    highcut,
                    CFG.sampling_rate,
                    order=self.rand_filter["order"],
                )

            # Add feature to output array
            X[:, n] = feat_val
            n += 1
            
        # Clip values in output array
        X = np.clip(X, -1024, 1024)

        # Handle potential NaN values
        X = np.nan_to_num(X, nan=0) / 32.0

        # Apply lowpass filter to entire output array
        X = butter_lowpass_filter(X, order=CFG.filter_order)

        # Initialize target probabilities array
        y_prob = np.zeros(CFG.target_size, dtype="float32")

        # If not in test mode, get target probabilities from the data row
        if self.mode != "test":
            y_prob = row[CFG.target_cols].values.astype(np.float32)

        # Return generated data and target probabilities
        return X, y_prob

# Helper functions

In [ ]:
class KLDivLossWithLogits(nn.KLDivLoss):
    """
    Custom KL divergence loss function with logits as input.

    This class inherits from the nn.KLDivLoss class but applies a log_softmax function
    to the input (y) before calculating the loss. This is because the KL divergence
    loss expects log probabilities as input, while the network typically outputs logits.
    """
    def __init__(self):
        super().__init__(reduction="batchmean")

    def forward(self, y, t):
        y = nn.functional.log_softmax(y, dim=1) # Apply log_softmax to y
        loss = super().forward(y, t)
        return loss


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def seed_torch(seed):
    """Seeds all relevant libraries for deterministic training."""
    
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True
    # pl.seed_everything(seed)

# Model

In [ ]:
class ResNet_1D_Block(nn.Module):
    """
    This class implements a basic residual block for 1D CNNs, which is a commonly
    used building block for image and time series classification tasks. It consists
    of two convolutional layers with batch normalization, activation functions (SiLU in this case),
    and dropout for regularization. A skip connection is added between the input and output
    of the block, allowing the network to learn identity mappings for better gradient flow.
    """
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        downsampling,
        dilation=1,
        groups=1,
        dropout=0.0,
    ):
        super(ResNet_1D_Block, self).__init__()

        # Initialize the block layers
        self.bn1 = nn.BatchNorm1d(num_features=in_channels)
        self.relu_1 = nn.Hardswish()
        self.relu_2 = nn.Hardswish()

        self.dropout = nn.Dropout(p=dropout, inplace=False)
        self.conv1 = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=False,
        )

        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=False,
        )

        self.maxpool = nn.MaxPool1d(
            kernel_size=2,
            stride=2,
            padding=0,
            dilation=dilation,
        )
        self.downsampling = downsampling

    def forward(self, x):
        # Apply the block layers to the input
        identity = x

        out = self.bn1(x)
        out = self.relu_1(out)
        out = self.dropout(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu_2(out)
        out = self.dropout(out)
        out = self.conv2(out)

        out = self.maxpool(out)
        identity = self.downsampling(x)

        out += identity
        return out


class EEGNet(nn.Module):
    """
    This class implements the EEGNet model, which is a compact and efficient
    convolutional neural network architecture for EEG classification tasks.
    The model consists of a series of convolutional layers with batch normalization,
    activation functions (SiLU in this case), and max pooling operations. The output
    of the convolutional layers is flattened and passed through a fully connected layer
    to obtain the final output. The model also includes a GRU layer for processing
    sequential data.
    """
    def __init__(
        self,
        kernels,
        in_channels,
        fixed_kernel_size,
        num_classes,
        linear_layer_features,
        dilation=1,
        groups=1,
    ):
        super(EEGNet, self).__init__()
        self.kernels = kernels
        self.planes = 24
        self.parallel_conv = nn.ModuleList()
        self.in_channels = in_channels

        for i, kernel_size in enumerate(list(self.kernels)):
            sep_conv = nn.Conv1d(
                in_channels=in_channels,
                out_channels=self.planes,
                kernel_size=(kernel_size),
                stride=1,
                padding=0,
                dilation=dilation,
                groups=groups,
                bias=False,
            )
            self.parallel_conv.append(sep_conv)

        # Initialize the model layers
        self.bn1 = nn.BatchNorm1d(num_features=self.planes)
        self.relu_1 = nn.SiLU()
        self.relu_2 = nn.SiLU()

        self.conv1 = nn.Conv1d(
            in_channels=self.planes,
            out_channels=self.planes,
            kernel_size=fixed_kernel_size,
            stride=2,
            padding=2,
            dilation=dilation,
            groups=groups,
            bias=False,
        )

        self.block = self._make_resnet_layer(
            kernel_size=fixed_kernel_size,
            stride=1,
            dilation=dilation,
            groups=groups,
            padding=fixed_kernel_size // 2,
        )
        self.bn2 = nn.BatchNorm1d(num_features=self.planes)
        self.avgpool = nn.AvgPool1d(kernel_size=6, stride=6, padding=2)

        # Initialize the GRU layer
        self.rnn = nn.GRU(
            input_size=self.in_channels,
            hidden_size=128,
            num_layers=1,
            bidirectional=True,
            # dropout=0.2,
        )

        # Initialize the linear layer
        self.fc = nn.Linear(in_features=linear_layer_features, out_features=num_classes)


    # Function to create a ResNet layer
    def _make_resnet_layer(
        self,
        kernel_size,
        stride,
        dilation=1,
        groups=1,
        blocks=9,
        padding=0,
        dropout=0.0,
    ):
        layers = []
        downsample = None
        base_width = self.planes

        for i in range(blocks):
            downsampling = nn.Sequential(
                nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
            )
            layers.append(
                ResNet_1D_Block(
                    in_channels=self.planes,
                    out_channels=self.planes,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    downsampling=downsampling,
                    dilation=dilation,
                    groups=groups,
                    dropout=dropout,
                )
            )
        return nn.Sequential(*layers)

    # Function to extract features from the input
    def extract_features(self, x):
        x = x.permute(0, 2, 1)

        out_sep = []
        for i in range(len(self.kernels)):
            sep = self.parallel_conv[i](x)
            out_sep.append(sep)

        out = torch.cat(out_sep, dim=2)
        out = self.bn1(out)
        out = self.relu_1(out)
        out = self.conv1(out)

        out = self.block(out)
        out = self.bn2(out)
        out = self.relu_2(out)
        out = self.avgpool(out)

        out = out.reshape(out.shape[0], -1)
        rnn_out, _ = self.rnn(x.permute(0, 2, 1))
        new_rnn_h = rnn_out[:, -1, :]

        new_out = torch.cat([out, new_rnn_h], dim=1)
        return new_out

    # Function to forward pass through the model
    def forward(self, x):
        new_out = self.extract_features(x)
        result = self.fc(new_out)
        return result

# Adan Optimizer

In [ ]:
class Adan(Optimizer):
    """
    Implements a pytorch variant of Adan
    Adan was proposed in
    Adan: Adaptive Nesterov Momentum Algorithm for Faster Optimizing Deep Models[J]. arXiv preprint arXiv:2208.06677, 2022.
    https://arxiv.org/abs/2208.06677
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining parameter groups.
        lr (float, optional): learning rate. (default: 1e-3)
        betas (Tuple[float, float, flot], optional): coefficients used for computing
            running averages of gradient and its norm. (default: (0.98, 0.92, 0.99))
        eps (float, optional): term added to the denominator to improve
            numerical stability. (default: 1e-8)
        weight_decay (float, optional): decoupled weight decay (L2 penalty) (default: 0)
        max_grad_norm (float, optional): value used to clip
            global grad norm (default: 0.0 no clip)
        no_prox (bool): how to perform the decoupled weight decay (default: False)
    """

    def __init__(
        self,
        params,
        lr=1e-3,
        betas=(0.98, 0.92, 0.99),
        eps=1e-8,
        weight_decay=0.2,
        max_grad_norm=0.0,
        no_prox=False,
    ):
        if not 0.0 <= max_grad_norm:
            raise ValueError("Invalid Max grad norm: {}".format(max_grad_norm))
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= betas[2] < 1.0:
            raise ValueError("Invalid beta parameter at index 2: {}".format(betas[2]))
        defaults = dict(
            lr=lr,
            betas=betas,
            eps=eps,
            weight_decay=weight_decay,
            max_grad_norm=max_grad_norm,
            no_prox=no_prox,
        )
        super(Adan, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(Adan, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault("no_prox", False)

    @torch.no_grad()
    def restart_opt(self):
        for group in self.param_groups:
            group["step"] = 0
            for p in group["params"]:
                if p.requires_grad:
                    state = self.state[p]
                    # State initialization

                    # Exponential moving average of gradient values
                    state["exp_avg"] = torch.zeros_like(p)
                    # Exponential moving average of squared gradient values
                    state["exp_avg_sq"] = torch.zeros_like(p)
                    # Exponential moving average of gradient difference
                    state["exp_avg_diff"] = torch.zeros_like(p)

    @torch.no_grad()
    def step(self):
        """
        Performs a single optimization step.
        """
        if self.defaults["max_grad_norm"] > 0:
            device = self.param_groups[0]["params"][0].device
            global_grad_norm = torch.zeros(1, device=device)

            max_grad_norm = torch.tensor(self.defaults["max_grad_norm"], device=device)
            for group in self.param_groups:

                for p in group["params"]:
                    if p.grad is not None:
                        grad = p.grad
                        global_grad_norm.add_(grad.pow(2).sum())

            global_grad_norm = torch.sqrt(global_grad_norm)

            clip_global_grad_norm = torch.clamp(
                max_grad_norm / (global_grad_norm + group["eps"]), max=1.0
            )
        else:
            clip_global_grad_norm = 1.0

        for group in self.param_groups:
            beta1, beta2, beta3 = group["betas"]
            # assume same step across group now to simplify things
            # per parameter step can be easily support by making it tensor, or pass list into kernel
            if "step" in group:
                group["step"] += 1
            else:
                group["step"] = 1

            bias_correction1 = 1.0 - beta1 ** group["step"]
            bias_correction2 = 1.0 - beta2 ** group["step"]
            bias_correction3 = 1.0 - beta3 ** group["step"]

            for p in group["params"]:
                if p.grad is None:
                    continue

                state = self.state[p]
                if len(state) == 0:
                    state["exp_avg"] = torch.zeros_like(p)
                    state["exp_avg_sq"] = torch.zeros_like(p)
                    state["exp_avg_diff"] = torch.zeros_like(p)

                grad = p.grad.mul_(clip_global_grad_norm)
                if "pre_grad" not in state or group["step"] == 1:
                    state["pre_grad"] = grad

                copy_grad = grad.clone()

                exp_avg, exp_avg_sq, exp_avg_diff = (
                    state["exp_avg"],
                    state["exp_avg_sq"],
                    state["exp_avg_diff"],
                )
                diff = grad - state["pre_grad"]

                update = grad + beta2 * diff
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)  # m_t
                exp_avg_diff.mul_(beta2).add_(diff, alpha=1 - beta2)  # diff_t
                exp_avg_sq.mul_(beta3).addcmul_(update, update, value=1 - beta3)  # n_t

                denom = ((exp_avg_sq).sqrt() / math.sqrt(bias_correction3)).add_(
                    group["eps"]
                )
                update = (
                    (
                        exp_avg / bias_correction1
                        + beta2 * exp_avg_diff / bias_correction2
                    )
                ).div_(denom)

                if group["no_prox"]:
                    p.data.mul_(1 - group["lr"] * group["weight_decay"])
                    p.add_(update, alpha=-group["lr"])
                else:
                    p.add_(update, alpha=-group["lr"])
                    p.data.div_(1 + group["lr"] * group["weight_decay"])

                state["pre_grad"] = copy_grad

# Train Func

In [ ]:
def train_fn(
    stage, fold, train_loader, model, criterion, optimizer, epoch, scheduler, device
):
    """
    This function performs one epoch of training on the provided dataset.
    It iterates through the training data loader, calculates the loss for each batch,
    performs backpropagation, updates the model weights using the optimizer,
    and tracks various metrics like average loss and learning rate.
    """
    model.train()

    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0

    for step, batch in enumerate(train_loader):
        eegs = batch["eeg"].to(device)
        labels = batch["labels"].to(device)
        batch_size = labels.size(0)

        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(eegs)
            # Calculate loss using log_softmax and the provided criterion
            loss = criterion(F.log_softmax(y_preds, dim=1), labels)

        if CFG.gradient_accumulation_steps > 1:
            # Gradient accumulation (accumulate gradients for multiple batches before update)
            loss = loss / CFG.gradient_accumulation_steps

        losses.update(loss.item(), batch_size)

        # Perform backward pass and gradient scaling (if mixed precision)
        scaler.scale(loss).backward()

        # Clip gradients to prevent exploding gradients
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)

        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            # Perform optimizer step and update scaler after gradient accumulation steps
            scaler.step(optimizer)
            scaler.update()
            global_step += 1
            if CFG.batch_scheduler: # Update learning rate scheduler if configured
                scheduler.step()
        end = time.time() # Update end time for logging

        if CFG.log_show and (
            step % CFG.log_step == 0 or step == (len(train_loader) - 1)
        ):
            # remain=timeSince(start, float(step + 1) / len(train_loader))
            LOGGER.info(
                f"Epoch {epoch+1} [{step}/{len(train_loader)}] Loss: {losses.val:.4f} Loss Avg:{losses.avg:.4f}"
            )
            # "Elapsed {remain:s} Grad: {grad_norm:.4f}  LR: {cheduler.get_lr()[0]:.8f}"

        if CFG.wandb:
            wandb.log(
                {
                    f"[fold{fold}] loss": losses.val,
                    f"[fold{fold}] lr": scheduler.get_lr()[0],
                }
            )
    return losses.avg

# Valid Func

In [ ]:
def valid_fn(stage, epoch, valid_loader, model, criterion, device):
    """
    This function iterates through the validation data loader,
    evaluates the model's performance on the validation set,
    calculates the average validation loss, and returns the loss and predictions.
    """
    losses = AverageMeter()  # Object to track and store average loss
    model.eval()  # Set the model to evaluation mode (disable dropout etc.)

    preds = []  # List to store model predictions
    targets = []  # List to store ground truth labels

    start = time.time()  # Record validation start time
    end = start  # Initialize end time (updated later)

    for step, batch in enumerate(valid_loader):
        eegs = batch["eeg"].to(device)
        labels = batch["labels"].to(device)
        batch_size = labels.size(0)

        with torch.no_grad(): 
            # Disable gradient calculation for validation (saves memory)
            y_preds = model(eegs)
            # Calculate loss using log_softmax and the provided criterion
            loss = criterion(F.log_softmax(y_preds, dim=1), labels)

        if CFG.gradient_accumulation_steps > 1:
            # Gradient accumulation (if used during training, adjust loss here)
            loss = loss / CFG.gradient_accumulation_steps

        losses.update(loss.item(), batch_size) # Update average loss meter

        # Collect predictions and targets for later evaluation metrics
        preds.append(nn.Softmax(dim=1)(y_preds).to("cpu").numpy())
        targets.append(labels.to("cpu").numpy())

        end = time.time()  # Update end time for logging

        if CFG.log_show and (
            step % CFG.log_step == 0 or step == (len(valid_loader) - 1)
        ):
            LOGGER.info(
                f"Epoch {epoch+1} VALIDATION: [{step}/{len(valid_loader)}] Val Loss: {losses.val:.4f} Val Loss Avg: {losses.avg:.4f}"
            )
    
    # Concatenate predictions and targets from all batches
    predictions = np.concatenate(preds)
    targets = np.concatenate(targets)

    return losses.avg, predictions

# Build Optimizer

In [ ]:
def build_optimizer(cfg, model, device, epochs, num_batches_per_epoch):
    """
    This function reads the optimizer configuration from the provided dictionary
    and creates the corresponding optimizer object. It supports various optimizers
    like SGD, Adam, AdamW, Ranger21, Lion, Adan, and SAM (Sharpness-Aware Minimization).
    The optimizer is configured with the learning rate, weight decay, and other
    parameters specified in the configuration.
    """
    lr = cfg.lr  # Extract learning rate from the configuration
    if cfg.optimizer == "SAM":
        # Build SAM optimizer (Sharpness-Aware Minimization)
        base_optimizer = (
            torch.optim.SGD
        )
        optimizer_model = SAM(
            model.parameters(),
            base_optimizer,
            lr=lr,
            momentum=0.9,
            weight_decay=cfg.weight_decay,
            adaptive=True,
        )
    elif cfg.optimizer == "Ranger21":
        # Build Ranger21 optimizer
        optimizer_model = Ranger21(
            model.parameters(),
            lr=lr,
            weight_decay=cfg.weight_decay,
            num_epochs=epochs,
            num_batches_per_epoch=num_batches_per_epoch,
        )
    elif cfg.optimizer == "SGD":
        # Build SGD optimizer
        optimizer_model = torch.optim.SGD(
            model.parameters(), lr=lr, weight_decay=cfg.weight_decay, momentum=0.9
        )
    elif cfg.optimizer == "Adam":
        # Build Adam optimizer
        optimizer_model = Adam(model.parameters(), lr=lr, weight_decay=CFG.weight_decay)
    elif cfg.optimizer == "AdamW":
        # Build AdamW optimizer
        optimizer_model = AdamW(
            model.parameters(), lr=lr, weight_decay=CFG.weight_decay
        )
    elif cfg.optimizer == "Lion":
        # Build Lion optimizer
        optimizer_model = Lion(model.parameters(), lr=lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == "Adan":
        # Build Adan optimizer
        optimizer_model = Adan(model.parameters(), lr=lr, weight_decay=cfg.weight_decay)
    else:
        raise NotImplementedError(f"Optimizer {cfg.optimizer} not supported!")

    return optimizer_model

# Scheduler

In [ ]:
def get_scheduler(optimizer, epochs, steps_per_epoch):
    """
    This function creates a learning rate scheduler based on the provided configuration.
    It supports various schedulers like ReduceLROnPlateau, CosineAnnealingLR,
    CosineAnnealingWarmRestarts, and OneCycleLR. The scheduler is configured with
    the optimizer, learning rate, and other parameters specified in the configuration.
    """
    if CFG.scheduler == "ReduceLROnPlateau":
        scheduler = ReduceLROnPlateau(optimizer, **CFG.reduce_params)
    elif CFG.scheduler == "CosineAnnealingLR":
        scheduler = CosineAnnealingLR(optimizer, **CFG.cosanneal_params)
    elif CFG.scheduler == "CosineAnnealingWarmRestarts":
        scheduler = CosineAnnealingWarmRestarts(optimizer, **CFG.cosanneal_res_params)
    elif CFG.scheduler == "OneCycleLR":
        scheduler = OneCycleLR(
            optimizer=optimizer,
            epochs=epochs,
            pct_start=0.0,
            steps_per_epoch=steps_per_epoch,
            max_lr=CFG.lr,
            div_factor=25,
            final_div_factor=4.0e-01,
        )
    return scheduler

# Train Loop

In [ ]:
def train_loop(stage, epochs, folds, fold, directory, prev_dir, eggs):
    """
    This function performs the following steps for a single fold in a k-fold
    cross-validation setting:

    1. Splits the data into training and validation sets based on the fold.
    2. Creates training and validation data loaders.
    3. Initializes the model (potentially loading weights from previous stages).
    4. Defines the optimizer, scheduler, and loss criterion.
    5. Trains the model for the specified number of epochs.
    6. Validates the model on the validation set after each epoch.
    7. Tracks the best validation loss and saves the model with the best performance.
    8. Returns the updated validation folds data and the best validation loss.
    """
    train_folds = folds[folds["fold"] != fold].reset_index(drop=True)
    valid_folds = folds[folds["fold"] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values

    # Creating training dataset
    train_dataset = EEGDataset(
        train_folds,
        batch_size=CFG.batch_size,
        mode="train",
        eegs=eggs,
        bandpass_filter=CFG.bandpass_filter,
        rand_filter=CFG.rand_filter,
    )
    
    # Creating validation dataset
    valid_dataset = EEGDataset(
        valid_folds,
        batch_size=CFG.batch_size,
        mode="valid",
        eegs=eggs,
        bandpass_filter=CFG.bandpass_filter,
    )

    # Creating training and validation data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=CFG.num_workers,
        pin_memory=True,
        drop_last=True,
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CFG.batch_size * CFG.batch_koef_valid,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True,
        drop_last=False,
    )

    LOGGER.info(
        f"========== stage: {stage} fold: {fold} training {len(train_loader)} / {len(valid_loader)} =========="
    )

    # Initialize the model
    model = EEGNet(
        kernels=CFG.kernels,
        in_channels=CFG.in_channels,
        fixed_kernel_size=CFG.fixed_kernel_size,
        num_classes=CFG.target_size,
        linear_layer_features=CFG.linear_layer_features,
    )

    # Load model weights from previous stage if available
    if stage > 1:
        model_weight = f"{prev_dir}{CFG.model_name}_ver-{CFG.VERSION}_stage-{stage-1}_fold-{fold}_best.pth"
        checkpoint = torch.load(model_weight, map_location=device)
        model.load_state_dict(checkpoint["model"])

    model.to(device)

    # CPMP: wrap the model to use all GPUs
    if CFG.parallel:
        model = nn.DataParallel(model)

    # Initialize the optimizer, scheduler, and loss criterion
    optimizer = build_optimizer(
        CFG, model, device, epochs=epochs, num_batches_per_epoch=len(train_loader)
    )
    scheduler = get_scheduler(
        optimizer, epochs=epochs, steps_per_epoch=len(train_loader)
    )
    criterion = nn.KLDivLoss(reduction="batchmean")

    # Train the model for the specified number of epochs
    best_score = np.inf
    for epoch in range(epochs):
        start_time = time.time()

        # train
        avg_loss = train_fn(
            stage,
            fold,
            train_loader,
            model,
            criterion,
            optimizer,
            epoch,
            scheduler,
            device,
        )

        # eval
        valid_dataset.set_offset(CFG.sample_offset)
        avg_val_loss, predictions = valid_fn(
            stage,
            epoch,
            valid_loader,
            model,
            criterion,
            device,
        )
        
        avg_loss_line = ''

        # multi-validation
        if CFG.multi_validation:
            multi_avg_val_loss = np.zeros(CFG.n_split_samples)
            start = (2 * CFG.sample_delta) // CFG.n_split_samples
            finish = (3 * CFG.sample_delta) // CFG.n_split_samples
            delta = (finish - start) // 5
            for i in range(CFG.n_split_samples):
                valid_dataset.set_offset(start)
                multi_avg_val_loss[i], _ = valid_fn(
                    stage,
                    epoch,
                    valid_loader,
                    model,
                    criterion,
                    device,
                )
                avg_loss_line += f" {multi_avg_val_loss[i]:.4f}"
                start += delta
            avg_loss_line += f" mean={np.mean(multi_avg_val_loss):.4f}"

        elapsed = time.time() - start_time

        LOGGER.info(
            f"Epoch {epoch+1} Avg Train Loss: {avg_loss:.4f} Avg Valid Loss: {avg_val_loss:.4f} / {avg_loss_line}"
        )
        #   time: {elapsed:.0f}s
        if CFG.wandb:
            wandb.log(
                {
                    f"[fold{fold}] stage": stage,
                    f"[fold{fold}] epoch": epoch + 1,
                    f"[fold{fold}] avg_train_loss": avg_loss,
                    f"[fold{fold}] avg_val_loss": avg_val_loss,
                    #f"[fold{fold}] score": score,
                }
            )

        # Save the model with the best validation loss
        if CFG.save_all_models:
            torch.save(
                {"model": model.module.state_dict(), "predictions": predictions},
                f"{directory}{CFG.model_name}_ver-{CFG.VERSION}_stage-{stage}_fold-{fold}_epoch-{epoch}_val-{avg_val_loss:.4f}_train-{avg_loss:.4f}.pth",
            )

        if best_score > avg_val_loss:
            best_score = avg_val_loss
            LOGGER.info(f"Epoch {epoch+1} Save Best Valid Loss: {avg_val_loss:.4f}")
            # CPMP: save the original model. It is stored as the module attribute of the DP model.
            torch.save(
                {"model": model.module.state_dict(), "predictions": predictions},
                f"{directory}{CFG.model_name}_ver-{CFG.VERSION}_stage-{stage}_fold-{fold}_best.pth",
            )

    # Load best model to use it for inference
    predictions = torch.load(
        f"{directory}{CFG.model_name}_ver-{CFG.VERSION}_stage-{stage}_fold-{fold}_best.pth",
        map_location=torch.device("cpu"),
    )["predictions"]

    # valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions
    valid_folds[CFG.pred_cols] = predictions
    valid_folds[CFG.target_cols] = valid_labels

    torch.cuda.empty_cache()
    gc.collect()

    # Return updated validation folds data and best validation loss
    return valid_folds, best_score

# Load train data

In [ ]:
# Read the training data
train = pd.read_csv(CFG.file_train)
TARGETS = train.columns[-6:]
print("Train shape:", train.shape)
print("Targets", list(TARGETS))

train["total_evaluators"] = train[CFG.target_cols].sum(axis=1)

train_uniq = train.drop_duplicates(subset=["eeg_id"] + list(TARGETS))

# Display some basic information about the training data
print(f"There are {train.patient_id.nunique()} patients in the training data.")
print(f"There are {train.eeg_id.nunique()} EEG IDs in the training data.")
print(f"There are {train_uniq.shape[0]} unique eeg_id + votes in the training data.")

# Display the distribution of the number of unique votes per EEG ID
if CFG.visualize:
    train_uniq.eeg_id.value_counts().value_counts().plot(
        kind="bar",
        title=f"Distribution of Count of EEG w Unique Vote: "
        f"{train_uniq.shape[0]} examples",
    )

del train_uniq
_ = gc.collect()

In [ ]:
# Visualize the distribution based on the number of total evaluators
if CFG.visualize:
    plt.figure(figsize=(10, 6))
    plt.hist(train["total_evaluators"], bins=10, color="blue", edgecolor="black")
    plt.title("Histogram of Total Evaluators")
    plt.xlabel("Total Evaluators")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

# Read the test data
tst_eeg_df = pd.read_parquet(CFG.file_features_test)
tst_eeg_features = tst_eeg_df.columns
print(f"There are {len(tst_eeg_features)} raw eeg features")
print(list(tst_eeg_features))
del tst_eeg_df
_ = gc.collect()

# Split Data

In [ ]:
# %%time
# Load EEG specifications
all_eeg_specs = np.load(CFG.file_eeg_specs, allow_pickle=True).item()

# Filter training data based on EEG specifications
train = train[train["label_id"].isin(all_eeg_specs.keys())].copy()
print(train.shape[0])

# Prepare target data with regularization
y_data = train[TARGETS].values + 0.166666667  # Regularization value
y_data = y_data / y_data.sum(axis=1, keepdims=True)
train[TARGETS] = y_data

# Add a new column "target" containing the expert consensus
train["target"] = train["expert_consensus"]

train[train['total_evaluators'] == CFG.test_total_eval].groupby(['expert_consensus','total_evaluators']).count()

In [ ]:
if CFG.test_total_eval > 0:
    # Assign a unique key identifier to each training sample
    train['key_id'] = range(train.shape[0])

    # List to store data subsets for different total_evaluators ranges (old)
    train_pop_olds = []

    # Loop through different total_evaluators ranges defined in CFG.total_evals_old
    for total_eval in CFG.total_evals_old:
        if type(total_eval) is list:
            pop_idx = (train["total_evaluators"] >= total_eval[0][0]) & (
                train["total_evaluators"] < total_eval[0][1]
            ) | (train["total_evaluators"] >= total_eval[1][0]) & (
                train["total_evaluators"] < total_eval[1][1]
            )
        else:
            pop_idx = (train["total_evaluators"] >= total_eval[0]) & (
                train["total_evaluators"] < total_eval[1]
            )

        # Filter training data based on the indexing and create a copy
        train_pop = train[pop_idx].copy().reset_index()

        # Initialize stratified group k-fold cross-validation (considering patient_id)
        sgkf = GroupKFold(n_splits=CFG.n_fold)
        train_pop["fold"] = -1
        for fold_id, (_, val_idx) in enumerate(
            sgkf.split(train_pop, y=train_pop["target"], groups=train_pop["patient_id"])
        ):
            train_pop.loc[val_idx, "fold"] = fold_id

        # Append the filtered and fold-assigned data subset to the list
        train_pop_olds.append(train_pop)
        print(train_pop.shape[0])

In [ ]:
# List to store data subsets for different total_evaluators configurations
train_pops = []
for eval_list in CFG.total_evaluators:
    # List to store filtered subsets based on individual configurations within the list
    result = []
    # Start with the entire training data for subsetting
    train_pop = train.copy()
    
    # Loop through each evaluation dictionary within the current configuration list
    for eval_dict in eval_list:
        # Extract the band range from the dictionary
        band = eval_dict['band']

        # Create boolean indexing to filter based on total_evaluators range
        pop_idx = (train_pop["total_evaluators"] >= band[0]) 
        pop_idx &= (train_pop["total_evaluators"] <= band[1])

        # Loop through evaluators to exclude (if any)
        for exclude in eval_dict['excl_evals']:
            pop_idx &= ~(train_pop['expert_consensus'] == exclude)
            pass

        # Filter the training data based on the current configuration and append to the result list
        result.append(train_pop[pop_idx])

    # Concatenate the filtered subsets from the current configuration list
    train_pop = pd.concat(result).copy().reset_index()

    # Initialize stratified group k-fold cross-validation (considering patient_id)
    sgkf = GroupKFold(n_splits=CFG.n_fold)

    # Add a new column "fold" to assign folds for cross-validation within the subset
    train_pop["fold"] = -1

    for fold_id, (_, val_idx) in enumerate(
        sgkf.split(train_pop, y=train_pop["target"], groups=train_pop["patient_id"])
    ):
        train_pop.loc[val_idx, "fold"] = fold_id

    # Append the filtered and fold-assigned data subset to the list
    train_pops.append(train_pop)

    # Print the number of samples in the current subset
    print(train_pop.shape[0])

train_0 = train_pops[0]
train_0[train_0['total_evaluators'] == CFG.test_total_eval].groupby(['expert_consensus','total_evaluators']).count()

In [ ]:
if CFG.test_total_eval > 0:
    df_old = train_pop_olds[0].copy(deep=True).set_index(['key_id'], drop=True).drop(columns=['fold'])
    df_new = train_pops[0].copy(deep=True).set_index(['key_id'], drop=True).drop(columns=['fold'])

    #outer merge the two DataFrames, adding an indicator column called 'Exist'
    diff_df = pd.merge(df_old, df_new, how='outer', indicator='Exist')

    #find which rows don't exist in both DataFrames
    diff_df = diff_df.loc[diff_df['Exist'] != 'both']
    display(diff_df)

    del df_old, df_new, diff_df, train_pop_olds
    _ = gc.collect()

In [ ]:
if CFG.visualize:
    print("Pop 1: train unique eeg_id + votes shape:", train_pops[0].shape)
    plt.figure(figsize=(10, 6))
    plt.hist(train["total_evaluators"], bins=10, color="blue", edgecolor="black")
    plt.title("Histogram of Total Evaluators")
    plt.xlabel("Total Evaluators")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

del all_eeg_specs
_ = gc.collect()

# Deduplicate Train EEG Id

In [ ]:
# %%time
if CFG.create_eegs:
    all_eegs = {}  # Dictionary to store loaded EEG data
    visualize = 1 if CFG.visualize else 0  # Set visualization flag based on configuration
    eeg_ids = train.eeg_id.unique()  # Get unique EEG IDs from the training data

    for i, eeg_id in tqdm(enumerate(eeg_ids)):  # Loop through unique EEG IDs with progress bar
        eeg_path = CFG.path_train / f"{eeg_id}.parquet"  # Construct path to the EEG file

        # Load EEG data from parquet file
        data = eeg_from_parquet(eeg_path, display=i < visualize)
        all_eegs[eeg_id] = data  # Store loaded data in the dictionary

        if i == visualize:  # After processing a certain number for visualization (if set)
            if CFG.create_eegs:  # If creating EEG data
                print(f"Processing {train['eeg_id'].nunique()} eeg parquets... ", end="")
            else:  # If loading pre-created EEG data
                print(f"Reading {len(eeg_ids)} eeg NumPys from disk.")
                break  # Exit the loop after processing the desired number for visualization

    # Save the dictionary containing all EEG data to a NumPy file
    np.save("./eegs", all_eegs)
else:  # If not creating EEG data, load from a pre-existing file
    all_eegs = np.load(CFG.file_raw_eeg, allow_pickle=True).item()  # Load EEG data from file

if CFG.visualize:  # Check if visualization flag is set
    frequencies = [1, 2, 4, 8, 16][::-1]  # Define list of frequencies for filtering (Hz)
    x = [all_eegs[eeg_ids[0]][:, 0]]  # Select the first EEG feature from the first EEG

    # Apply butter low-pass filter to the selected feature with different frequencies
    for frequency in frequencies:
        x.append(butter_lowpass_filter(x[0], cutoff_freq=frequency))

    # Create a plot to visualize the filter effects
    plt.figure(figsize=(12, 8))
    plt.plot(range(CFG.nsamples), x[0], label="without filter")
    for k in range(1, len(x)):
        # Shift each filtered signal vertically to avoid overlapping lines
        plt.plot(
            range(CFG.nsamples),
            x[k] - k * (x[0].max() - x[0].min()),
            label=f"with filter {frequencies[k - 1]}Hz",
        )

    plt.legend()
    plt.yticks([])
    plt.title("Butter Low-Pass Filter Examples", size=18)
    plt.show()

In [ ]:

if CFG.visualize:
    # Create an EEG dataset object from the first element of train_pops
    train_dataset = EEGDataset(
        train_pops[0], batch_size=CFG.batch_size, eegs=all_eegs, mode="train"
    )

    # Create a data loader for efficient batch processing
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,  # Don't shuffle data for visualization
        num_workers=CFG.num_workers,
        pin_memory=True,  # Pin data to GPU memory for faster access (if applicable)
        drop_last=True,  # Drop the last incomplete batch if necessary
    )

    output = train_dataset[0]
    X, y = output["eeg"], output["labels"]
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    # Create a random tensor as input for model testing (replace with real input later)
    iot = torch.randn(2, CFG.nsamples, CFG.in_channels)  # .cuda()

    # Define the EEGNet model architecture
    model = EEGNet(
        kernels=CFG.kernels,
        in_channels=CFG.in_channels,
        fixed_kernel_size=CFG.fixed_kernel_size,
        num_classes=CFG.target_size,
        linear_layer_features=CFG.linear_layer_features,
    )

    # Test the model output with the random tensor
    output = model(iot)
    print(f"Model output shape: {output.shape}")

    # Loop through a batch of data from the data loader
    for batch in train_loader:
        # Extract the EEG data (X) and labels (y) from the batch
        X = batch.pop("eeg")
        y = batch.pop("labels")

        # Visualize the first 4 samples in the batch
        for item in range(4):
            plt.figure(figsize=(20, 4))
            offset = 0
            for col in range(X.shape[-1]):
                # Avoid overlapping lines by adjusting offset based on min/max values
                if col != 0:
                    offset -= X[item, :, col].min()
                plt.plot(
                    range(CFG.nsamples),
                    X[item, :, col] + offset,
                    label=f"feature {col+1}",
                )
                offset += X[item, :, col].max()

            # Format and display the target labels
            tt = f"{y[col][0]:0.1f}"
            for t in y[col][1:]:
                tt += f", {t:0.1f}"
            plt.title(f"EEG_Id = {eeg_ids[item]}\nTarget = {tt}", size=14)
            plt.legend()
            plt.show()

        # Break out of the loop after visualizing 4 samples
        break

    # Delete temporary tensors and model to free memory
    del iot, model
    gc.collect()

# Train Stages

In [ ]:
def get_score(preds, targets):
    """
    Calculates the score (metric) between predicted and target values.
    """

    # Create a copy of the predictions
    oof = pd.DataFrame(preds.copy())

    # Add an "id" column with sequential indices for each data point in predictions
    oof["id"] = np.arange(len(oof))

    # Create a copy of the target
    true = pd.DataFrame(targets.copy())

    # Add an "id" column with sequential indices for each data point in targets
    true["id"] = np.arange(len(true))

    # Calculate the score (metric) using the `kaggle_kl_div` function
    cv = kaggle_kl_div.score(solution=true, submission=oof, row_id_column_name="id")

    # Return the calculated score
    return cv


def get_result(result_df):
    """
    Processes and evaluates the final results DataFrame.
    """

    # Select columns from the result DataFrame containing EEG IDs and target columns as defined in CFG
    gt = result_df[["eeg_id"] + CFG.target_cols]

    # Sort the data by "eeg_id" to ensure consistent order
    gt.sort_values(by="eeg_id", inplace=True)

    # Reset the index after sorting
    gt.reset_index(inplace=True, drop=True)

    # Select columns from the result DataFrame containing EEG IDs and predicted target columns
    preds = result_df[["eeg_id"] + CFG.pred_cols]

    # Rename the predicted target columns to match the target columns format
    preds.columns = ["eeg_id"] + CFG.target_cols

    # Sort the data by "eeg_id" to ensure consistent order (same as ground truth)
    preds.sort_values(by="eeg_id", inplace=True)

    # Reset the index after sorting
    preds.reset_index(inplace=True, drop=True)

    # Calculate the score using the `get_score` function
    score_loss = get_score(gt[CFG.target_cols], preds[CFG.target_cols])

    # Log the score information
    LOGGER.info(f"Score with best loss weights: {score_loss}")


In [ ]:
if __name__ == "__main__" and CFG.train_by_stages:
    # Check if script is executed directly and training by stages is enabled

    # Set random seed for reproducibility
    seed_torch(seed=CFG.seed)

    # Variable to store the directory path from the previous stage
    prev_dir = ""

    # Loop through all defined stages
    for stage in range(len(CFG.total_evaluators)):
        # Construct directory path for the current stage outputs
        pop_dir = f"{OUTPUT_DIR}pop_{stage+1}_weight_oof/"
        # Create the directory if it doesn't exist
        if not os.path.exists(pop_dir):
            os.makedirs(pop_dir)

        # Skip stages not included in the training stages list (potentially for fine-tuning)
        if stage not in CFG.train_stages:
            prev_dir = pop_dir
            continue

        # Initialize empty DataFrame to store out-of-fold (OOF) predictions
        oof_df = pd.DataFrame()
        # Initialize empty list to store scores for each fold
        scores = []

        # Loop through each fold in the defined training folds
        for fold in CFG.train_folds:
            # Call the `train_loop` function to perform training for the current stage, fold, etc.
            train_oof_df, score = train_loop(
                stage=stage + 1,
                epochs=CFG.epochs[stage],
                fold=fold,
                folds=train_pops[stage],
                directory=pop_dir,
                prev_dir=prev_dir,
                eggs=all_eegs,
            )

            # Concatenate OOF predictions from the current fold to the main OOF DataFrame
            oof_df = pd.concat([oof_df, train_oof_df])
            # Append the score for the current fold
            scores.append(score)

            # Log information about the fold result
            LOGGER.info(f"========== stage: {stage+1} fold: {fold} result ==========")
            LOGGER.info(f"Score with best loss weights stage{stage+1}: {score:.4f}")

        # Log overall CV (Cross-Validation) score (average of fold scores)
        LOGGER.info(f"==================== CV ====================")
        LOGGER.info(f"Score with best loss weights: {np.mean(scores):.4f}")

        # Reset index of the OOF DataFrame
        oof_df.reset_index(drop=True, inplace=True)

        # Save the OOF DataFrame to a CSV file with specific naming convention
        oof_df.to_csv(
            f"{pop_dir}{CFG.model_name}_oof_df_ver-{CFG.VERSION}_stage-{stage+1}.csv",
            index=False,
        )

        # Update the previous directory path for referencing in subsequent stages
        prev_dir = pop_dir

    # If using Weights & Biases logging, finish the session
    if CFG.wandb:
        wandb.finish()


In [ ]:
if __name__ == "__main__" and CFG.train_by_folds:
    # Check if script is executed directly and training by folds is enabled

    # Set random seed for reproducibility
    seed_torch(seed=CFG.seed)

    # Initialize dictionaries to store scores and OOF DataFrames for each stage
    stages_scores = {i: [] for i in CFG.train_stages}
    stages_oof_df = {i: pd.DataFrame() for i in CFG.train_stages}

    # Loop through each fold in the defined training folds
    for fold in CFG.train_folds:

        # Variable to store the directory path from the previous stage
        prev_dir = ""

        # Loop through all defined stages
        for stage in range(len(CFG.total_evaluators)):
            # Construct directory path for the current stage outputs
            pop_dir = f"{OUTPUT_DIR}pop_{stage+1}_weight_oof/"
            # Create the directory if it doesn't exist
            if not os.path.exists(pop_dir):
                os.makedirs(pop_dir)

            # Skip stages not included in the training stages list
            if stage not in CFG.train_stages:
                prev_dir = pop_dir
                continue

            # Call the `train_loop` function to perform training for the current stage, fold, etc.
            train_oof_df, score = train_loop(
                stage=stage + 1,
                epochs=CFG.epochs[stage],
                fold=fold,
                folds=train_pops[stage],
                directory=pop_dir,
                prev_dir=prev_dir,
                eggs=all_eegs,
            )

            # Concatenate OOF predictions from the current fold to the stage's OOF DataFrame
            stages_oof_df[stage] = pd.concat([stages_oof_df[stage], train_oof_df])
            # Append the score for the current fold to the stage's score list
            stages_scores[stage].append(score)

            # Update the previous directory path for referencing in subsequent stages
            prev_dir = pop_dir

            # Log information about the fold result
            LOGGER.info(f"========== fold: {fold} stage: {stage+1} result ==========")
            LOGGER.info(f"Score with best loss weights stage{stage+1}: {score:.4f}")

    # Log overall CV (Cross-Validation) score (average score) for each stage
    for stage, scores in stages_scores.items():
        LOGGER.info(f"============ CV score with best loss weights ============")
        LOGGER.info(f"Stage {stage}: {np.mean(scores):.4f}")

    # Save OOF DataFrames for each stage to separate CSV files
    for stage, oof_df in stages_oof_df.items():
        pop_dir = f"{OUTPUT_DIR}pop_{stage+1}_weight_oof/"
        # Reset index of the OOF DataFrame
        oof_df.reset_index(drop=True, inplace=True)
        # Save the OOF DataFrame to a CSV file with specific naming convention
        oof_df.to_csv(
            f"{pop_dir}{CFG.model_name}_oof_df_ver-{CFG.VERSION}_stage-{stage+1}.csv",
            index=False,
        )

    # If using Weights & Biases logging, finish the session
    if CFG.wandb:
        wandb.finish()


# Submission

In [ ]:
# === Pre-process OOF ===
# Prepare ground truth data
gt = oof_df[["eeg_id"] + CFG.target_cols]  # Select relevant columns from OOF DataFrame
gt.sort_values(by="eeg_id", inplace=True)  # Sort by "eeg_id" for consistent order
gt.reset_index(inplace=True, drop=True)  # Reset index after sorting

# Prepare predicted data
preds = oof_df[["eeg_id"] + CFG.pred_cols]  # Select predicted target columns
preds.columns = ["eeg_id"] + CFG.target_cols  # Rename columns to match target format
preds.sort_values(by="eeg_id", inplace=True)  # Sort by "eeg_id" for consistency
preds.reset_index(inplace=True, drop=True)  # Reset index after sorting

# Extract target values (ground truth and predicted)
y_trues = gt[CFG.target_cols]  # Select target columns from ground truth DataFrame
y_preds = preds[CFG.target_cols]  # Select target columns from predicted DataFrame

# Create DataFrames for calculating score (metric)
oof = pd.DataFrame(y_preds.copy())  # Copy predicted values to avoid modifying original DataFrame
oof["id"] = np.arange(len(oof))  # Add "id" column with sequential indices

true = pd.DataFrame(y_trues.copy())  # Copy ground truth values to avoid modifying original DataFrame
true["id"] = np.arange(len(true))  # Add "id" column with sequential indices

# Calculate CV (Cross-Validation) score using kaggle_kl_div library
cv = kaggle_kl_div.score(solution=true, submission=oof, row_id_column_name="id")

# Print the CV score with informative message
print(f"CV Score with resnet1D_gru Raw EEG = {cv:.4f}")
